<a href="https://colab.research.google.com/github/MaharshiYeluri01/SpeechRecognition/blob/master/Speech_recognition_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import os
from pathlib import Path

import librosa
import numpy as np
from hmmlearn import hmm
from librosa.feature import mfcc, chroma_stft

In [39]:
class HMMTrainer:
    def __init__(self, model_name='GaussianHMM', n_components=4, cov_type='diag', n_iter=1000):
        self.model_name = model_name
        self.n_components = n_components
        self.cov_type = cov_type
        self.n_iter = n_iter
        self.models = []

        if self.model_name == 'GaussianHMM':
            self.model = hmm.GaussianHMM(n_components=self.n_components, 
                    covariance_type=self.cov_type, n_iter=self.n_iter)
        else:
            raise TypeError('Invalid model type')

    # X is a 2D numpy array where each row is 13D
    def train(self, X):
        np.seterr(all='ignore')
        self.models.append(self.model.fit(X))

    # Run the model on input data
    def get_score(self, input_data):
        return self.model.score(input_data)

In [40]:
input_folder = Path('~/Downloads/hmm-speech-recognition-0.1/audio').expanduser()

In [41]:
for dirname in os.listdir(input_folder):
    # Get the name of the subfolder 
    subfolder = os.path.join(input_folder, dirname)
    #print(subfolder)
    label = subfolder[subfolder.rfind('/') + 1:]
    print(label)

apple
.DS_Store
kiwi
lime
banana
pineapple
orange
peach


In [42]:
hmm_models = []
for dirname in os.listdir(input_folder):
    subfolder = os.path.join(input_folder, dirname)
    if not os.path.isdir(subfolder): 
         continue
    label = subfolder[subfolder.rfind('/') + 1:]
    X = np.array([])
    y_words = []
    for filename in [x for x in os.listdir(subfolder) if x.endswith('.wav')][:-1]:
        filepath = os.path.join(subfolder, filename)
        y, sr = librosa.load(filepath)            
        mfcc_features = mfcc(y=y, sr=sr)
        if len(X) == 0:
            X = mfcc_features[:,:15]
        else:
            X = np.append(X, mfcc_features[:,:15], axis=0)            
        y_words.append(label)
    print('X.shape =', X.shape)
    hmm_trainer = HMMTrainer()
    hmm_trainer.train(X)
    hmm_models.append((hmm_trainer, label))
    hmm_trainer = None

X.shape = (280, 15)
X.shape = (280, 15)
X.shape = (280, 15)
X.shape = (280, 15)
X.shape = (280, 15)
X.shape = (280, 15)
X.shape = (280, 15)


In [43]:
# Test files
input_files = [
    input_folder / 'apple' / 'apple15.wav',
    input_folder / 'banana' / 'banana15.wav',
    input_folder / 'kiwi' / 'kiwi15.wav',
    input_folder / 'orange' / 'orange15.wav',
    input_folder / 'pineapple' / 'pineapple15.wav',
    input_folder / 'lime' / 'lime15.wav',
    input_folder / 'peach' / 'peach15.wav',
]

In [44]:
for input_file in input_files:
    y, sr = librosa.load(input_file)

    # Extract MFCC features
    mfcc_features = mfcc(y=y, sr=sr)
    mfcc_features = mfcc_features[:, :15]

    scores = []
    for item in hmm_models:
        hmm_model, label = item

        score = hmm_model.get_score(mfcc_features)
        scores.append(score)
    index = np.array(scores).argmax()
    print("\nTrue:", input_file.parts[-2])
    print("Predicted:", hmm_models[index][1])


True: apple
Predicted: apple

True: banana
Predicted: banana

True: kiwi
Predicted: kiwi

True: orange
Predicted: orange

True: pineapple
Predicted: pineapple

True: lime
Predicted: lime

True: peach
Predicted: peach
